In [ ]:
import tweepy
import json
import pandas as pd
from datetime import datetime
from datetime import timedelta
import preprocessor as pre
import regex as re


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# access the drive
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# the file you want to upload, here simple example
f = drive.CreateFile()
f.SetContentFile('testupload.txt')

# upload the file
f.Upload()
print('title: %s, mimeType: %s' % (f['title'], f['mimeType']))


def tweetprocess(tweet):
    #https://towardsdatascience.com/basic-tweet-preprocessing-in-python-efd8360d529e
    cleantweet = pre.clean(tweet)
    cleantweet = cleantweet.lower()
    cleantweet = re.sub("\d+", "", cleantweet)
    cleantweet = re.sub(r'[^\w\s]', '', cleantweet)   
    return cleantweet


def tweetquality(user_verified, favorite_count, retweet_count):
    if user_verified == True or favorite_count > 100 or retweet_count > 10:
        return True
    else:
        return False

cur_time_utc = datetime.utcnow().replace(microsecond=0)

# Add Twitter API key and secret
consumer_key = "juf1eoH4Gn5ZqCtsuKm7PUdAO"
consumer_secret = "7VX0jDqBKTAjVzvYiKhvW4b2tAebMc38FzmPrpcnILkJRt5tF5"


# Handling authentication with Twitter
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

# Create a wrapper for the Twitter API
api = tweepy.API(auth, wait_on_rate_limit=True)

# Helper function for handling pagination in our search and handle rate limits
def limit_handled(cursor):
   while True:
       try:
           yield cursor.next()
       except StopIteration:
           break
       except tweepy.errors:
           print('Reached rate limit. Sleeping for >15 minutes')
           time.sleep(15 * 61)

# Define the term you will be using for searching tweets

query = '$TLRY AND -$GME AND -$HOOD AND -$TSLA AND -$AMC'# OR $AMC OR $TLRY OR $HOOD OR $TSLA OR $GME'
query = query + '-filter:retweets' #Exclude retweets

until_date = '2022-04-03'
since_id = '1510044240487022595' #tweet at end of Apri 2th
#max_id = '1508594545335930000' #tweet at end of March 28th

# Define how many tweets to get from the Twitter API
count = 10000

# Let's search for tweets using Tweepy
search = limit_handled(tweepy.Cursor(api.search_tweets #https://docs.tweepy.org/en/stable/api.html#search-tweets
                       ,q=query
                       ,tweet_mode='extended'
                       ,lang='en'
                       ,result_type="recent"
                       #,since_id = since_id
                       #,max_id = max_id
                       ,until=until_date
                       ).items(count))

dftweets = pd.DataFrame()
for tweet in search:
    dftweets = dftweets.append(pd.json_normalize(tweet._json))
print('# tweets:',len(dftweets),'\n')

dftweets['full_text_preprocessed'] = dftweets.apply(lambda row : tweetprocess(row['full_text']), axis = 1)
dftweets['quality'] = dftweets.apply(lambda row : tweetquality(row['user.verified'], row['favorite_count'], row['retweet_count']), axis = 1)
dftweets['num_cashtags'] = dftweets.apply(lambda row : str(row['entities.symbols']).count('text'), axis = 1)

print(dftweets[['id','created_at','full_text','full_text_preprocessed','entities.symbols','num_cashtags']])

dftweets_tofile = dftweets[['id','created_at','full_text_preprocessed','entities.symbols','quality','num_cashtags']]

dftweets_tofile.to_csv('tweepy '+cur_time_utc.strftime("%m-%d-%Y %H%M%S")+'.csv', index=False)#,line_terminator = '\r')


# tweets: 4197 

                     id                      created_at  \
0   1510391647900684288  Sat Apr 02 23:00:05 +0000 2022   
0   1510386090284576768  Sat Apr 02 22:38:00 +0000 2022   
0   1510384407227498499  Sat Apr 02 22:31:18 +0000 2022   
0   1510384247571623937  Sat Apr 02 22:30:40 +0000 2022   
0   1510382856065130496  Sat Apr 02 22:25:09 +0000 2022   
..                  ...                             ...   
0   1508212726166097921  Sun Mar 27 22:41:49 +0000 2022   
0   1508212364533186564  Sun Mar 27 22:40:23 +0000 2022   
0   1508210995562684418  Sun Mar 27 22:34:57 +0000 2022   
0   1508210787956969480  Sun Mar 27 22:34:07 +0000 2022   
0   1508210431818493953  Sun Mar 27 22:32:42 +0000 2022   

                                            full_text  \
0   This year: \n$AAPL is down -4.1% \n$AMD is dow...   
0   $MSOS $TLRY $CGC SAFE banking is a huge cataly...   
0   $TLRY midterms comomg up. Senate will pass or ...   
0   $TLRY I forgot to remind everyone that at 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4bd80a34-af3c-4211-a433-b5e963efdc38' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>